## Система автоматичного розпізнавання патологій на знімках зубів

#### Команда
Топоровська Вікторія, Курилець Анастасія, Чеботарьова Юлія

#### Ідея
Cтворити модель, яка може дозволити автоматизувати процес діагностики зубів, і зробити цю процедуру доступнішою.


Дивлячись на можливі імплементації, для початкової демо версії ми вирішили використовувати Sequentia. Це є найпростіша модель типу CNN, з лінійною послідовністю.
Починаємо з завантажування бібліотек. 
(IMPORT може повторюватися, бо чомуся ipynb моментами ігнорує попередню ініціалізацію)

In [3]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import os
import numpy as np
import pandas as np

import matplotlib.pyplot as plt
%matplotlib inline

Грузимо наші фото. 

наша структура


--- data

    --- здорові
    --- хворі


Використовуємо ImageDataGenerator він автоматично зчитує та попередньо обробляє дані. 


rescale=1./255 - масштабуємо яскравість пікселя з діапазону [0-225] до [0, 1]. щоб модель працювала краще і швидше. 225 - найяскравіше можливе знчаення для пікселя

flow_from_directory() - щоб не підвантажувати зображення окремо з кожної папки.

target_size=(150, 150) — усі зображення масштабуються до одного розміру 150×150 пікселів. Це потрібно, тому що нейронна мережа приймає вхідні дані однакових розмірів. Якщо зображення різні за розміром — навчання буде неможливе.

batch_size=32 — визначає, скільки зображень обробляється за один крок. Малий batch робить навчання повільнішим, але точнішим; великий — швидшим, але менш стабільним. 32 — це збалансоване значення

class_mode='binary' - задає тип класифікації, у нашій початковій спробі це хворий чи здоровий

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #знову ж таки чомусь бачить з попередньої комірки через раз


train_dir = "data"  


train_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,              
    target_size=(150, 150), 
    batch_size=32,          
    class_mode='binary'     
)


Found 396 images belonging to 2 classes.


Так як у нас є просто дата сет з даними де є хворі і не хворі зуби, ми розділяємо на тренувальні і тестувальні дані. 75% ми даємо на тренування, а 25% на тестування, створюємо всі потрібні папки

In [5]:
import os
import shutil
import random


data_dir = "data"  
healthy_dir = os.path.join(data_dir, "DENTAL_HEALTHY_TEETH") 
diseased_dir = os.path.join(data_dir, "DENTAL_NOT_HEALTHY_TEETH") 


train_dir = "train"
test_dir = "test"

for folder in [train_dir, test_dir]:
    os.makedirs(os.path.join(folder, "healthy"), exist_ok=True)
    os.makedirs(os.path.join(folder, "diseased"), exist_ok=True)


def split_data(source_dir, train_dest, test_dest, train_ratio=0.75):
    files = os.listdir(source_dir)
    random.shuffle(files)
    n_train = int(len(files) * train_ratio)
    train_files = files[:n_train]
    test_files = files[n_train:]

    for f in train_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(train_dest, f))
    for f in test_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(test_dest, f))


split_data(
    healthy_dir,
    os.path.join(train_dir, "healthy"),
    os.path.join(test_dir, "healthy")
)


split_data(
    diseased_dir,
    os.path.join(train_dir, "diseased"),
    os.path.join(test_dir, "diseased")
)

print("Розподіл завершено")


Розподіл завершено


Також ми отримуємо дві папки з даними і тепер вже працюємо з нашою моделлю

epochs = 20 — це кількість повних проходів через увесь навчальний набір даних. Модель 20 разів побачить усі зображення з тренувального набору і щоразу оновлюватиме свої ваги.

batch_size = 16 — це кількість зображень, які мережа обробляє за один крок (перед оновленням ваг).

In [6]:
from tensorflow.keras import backend as K


img_width, img_height = 150, 150


train_data_dir = 'train'  
test_data_dir = 'test'   


epochs = 20
batch_size = 16


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


Створення моделі

Наша модель складається з кількох етапів обробки зображення. Перші три шари Conv2D (із 32, 32 та 64 фільтрами розміру 3×3) послідовно виділяють важливі ознаки з рентгенівських знімків зубів — від простих контурів і форм до складніших структур. Після кожного згорткового шару йде активація ReLU, що додає нелінійність і допомагає моделі виявляти складні візуальні патерни. MaxPooling2D після кожного блоку зменшує розмірність зображення, виділяючи найважливіші ознаки й знижуючи обчислювальну складність. Потім шар Flatten перетворює двовимірні ознаки в одновимірний вектор, який подається у Dense-шар із 64 нейронами для аналізу знайдених патернів. Dropout(0.5) випадково вимикає половину нейронів під час навчання, щоб запобігти перенавчанню. Нарешті, вихідний Dense(1) із активацією sigmoid видає ймовірність наявності патології — значення близьке до 1 означає патологію, а до 0 — норму.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

model = Sequential()


model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(1))
model.add(Activation('sigmoid'))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Компіляція

Компіляція це налаштування нашої моделі, тобто як саме вона буде вчитися, що саме буде оптимізувати і як оцінювати результати

loss='binary_crossentropy' — порівнює передбачене значення (ймовірність патології) із правильним класом (0 або 1) і вимірює, наскільки сильно модель помиляється

optimizer='rmsprop' — це алгоритм, який оновлює ваги моделі на основі градієнтів

metrics=['accuracy'] — вказує, наточність (accuracy), тобто частку правильних передбачень.

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


генератори даних

щоб наша модель не лякалася шумів, і зайвих поворотів розворотів, ми самостійно генеруємо якісь невеличкі повороти і шуми, щоб модель вчилася працювати і зними

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 370 images belonging to 2 classes.
Found 174 images belonging to 2 classes.


Навчання моделі

In [10]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = max(1, test_generator.samples // batch_size)  

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.4944 - loss: 0.8711 - val_accuracy: 0.5375 - val_loss: 0.6913
Epoch 2/20
 1/23 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.3750 - loss: 0.7063

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3750 - loss: 0.7063 - val_accuracy: 0.5500 - val_loss: 0.6896
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.5621 - loss: 0.6911 - val_accuracy: 0.5500 - val_loss: 0.7052
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5625 - loss: 0.7678 - val_accuracy: 0.4563 - val_loss: 0.6996
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 147ms/step - accuracy: 0.5226 - loss: 0.6978 - val_accuracy: 0.5375 - val_loss: 0.6928
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5625 - loss: 0.6883 - val_accuracy: 0.5625 - val_loss: 0.6859
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.5508 - loss: 0.6958 - val_accuracy: 0.5500 - val_loss: 0.6904
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5437 - val_loss: 0.6911
Epoch 9/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.5395 - loss: 0.6933 - val_accuracy: 0.5562 - val_loss

наша модель працює на угад +- що не дуже добре

збереження і оцінка

In [ ]:

model.save_weights('teeth_model.weights.h5')


scores = model.evaluate(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.5600 - loss: 0.6868

compile_metrics: 56.00%


Дає точість +- 50/50, що не дуже добре, так як наша модель може просто завчити, спробуємо надати менше поколінь


In [11]:
from tensorflow.keras import backend as K


img_width, img_height = 150, 150


train_data_dir = 'train'  
test_data_dir = 'test'   


epochs = 10
batch_size = 16

# Вхідна форма для моделі
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 370 images belonging to 2 classes.
Found 174 images belonging to 2 classes.


In [ ]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = max(1, test_generator.samples // batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps)


Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 154ms/step - accuracy: 0.5593 - loss: 0.6911 - val_accuracy: 0.5500 - val_loss: 0.6896
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6250 - loss: 0.6804 - val_accuracy: 0.5312 - val_loss: 0.6912
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 142ms/step - accuracy: 0.5537 - loss: 0.6896 - val_accuracy: 0.5250 - val_loss: 0.6925
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6875 - loss: 0.6643 - val_accuracy: 0.5562 - val_loss: 0.6870
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - accuracy: 0.5537 - loss: 0.6916 - val_accuracy: 0.5500 - val_loss: 0.6890
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6875 - loss: 0.6719 - val_accuracy: 0.5562 - val_loss: 0.6876
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.5650 - loss: 0.6887 - val_accuracy: 0.5500 - val_loss: 0.6884
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4375 - loss: 0.7014 - val_accuracy: 0.5312

In [15]:

model.save_weights('teeth_model_2.weights.h5')


scores = model.evaluate(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.5517 - loss: 0.6883

compile_metrics: 55.17%


In [17]:
from tensorflow.keras import backend as K


img_width, img_height = 150, 150


train_data_dir = 'train'  
test_data_dir = 'test'   


epochs = 100
batch_size = 16

# Вхідна форма для моделі
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


steps_per_epoch = train_generator.samples // batch_size
validation_steps = max(1, test_generator.samples // batch_size)  

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps)


model.save_weights('teeth_model_3.weights.h5')


scores = model.evaluate(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Found 370 images belonging to 2 classes.
Found 174 images belonging to 2 classes.
Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.5593 - loss: 0.6908 - val_accuracy: 0.5688 - val_loss: 0.6851
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6250 - loss: 0.6581 - val_accuracy: 0.5562 - val_loss: 0.6939
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.5537 - loss: 0.6888 - val_accuracy: 0.5625 - val_loss: 0.6855
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6875 - loss: 0.6478 - val_accuracy: 0.5375 - val_loss: 0.6936
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 142ms/step - accuracy: 0.5734 - loss: 0.6849 - val_accuracy: 0.5500 - val_loss: 0.6888
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.3750 - loss: 0.7670 - val_accuracy: 0.5562 - val_loss: 0.6865
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.5565 - loss: 0.6878 - val_accuracy: 0.5375 - val_loss: 0.6897
Epoch 8/100
23/23 

бачимо що модель все одно дуже погано працює

Одна з причин це те що у нас пащі з зубами які у різному стані. Тому варто розбивати на категорії і ідентифікувати хвороби вже по категоріях.

In [18]:
import os
import shutil
import random

base_dir = 'data_clas'


train_dir = 'train_clas'
test_dir = 'test_clas'


test_ratio = 0.75

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)

    if not os.path.isdir(class_path):
        continue

    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    random.shuffle(files)

    test_size = int(len(files) * test_ratio)
    test_files = files[:test_size]
    train_files = files[test_size:]

    for f in train_files:
        src = os.path.join(class_path, f)
        dst = os.path.join(train_dir, class_name, f)
        shutil.move(src, dst)

    for f in test_files:
        src = os.path.join(class_path, f)
        dst = os.path.join(test_dir, class_name, f)
        shutil.move(src, dst)


print(f"Тренувальні дані збережено")


Тренувальні дані збережено


In [19]:
train_dir = 'train_clas'
test_dir = 'test_clas'


img_height, img_width = 150, 150
batch_size = 32


In [20]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  
)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 106 images belonging to 6 classes.
Found 24 images belonging to 6 classes.
Found 387 images belonging to 6 classes.


In [21]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(train_generator.num_classes, activation='softmax')
])


In [25]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [26]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n Точність: {test_acc*100:.4f} %")

model.save('teeth_classifier.keras')  


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.0439 - loss: 1.8315  

 Точність: 4.3928 %


In [27]:
import numpy as np

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)


print("Класи:", test_generator.class_indices)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step
Класи: {'BDC-BDR': 0, 'Caries': 1, 'Fractured Teeth': 2, 'Healthy Teeth': 3, 'Impacted teeth': 4, 'Infection': 5}
